## Demo: getting weather data via Open Meteo (demo part 1)

### Open Meteo

Open Meteo is a weather API that collates both forecast and historical weather data globally. It does this by working with 9 different government weather organisations, making data access consistent and reliable.

THis notebook demonstrates how to source the same weather metrics that are provided by EOS' weather API from Open Meteo directly. This option is more flexible that using the EOS API, as we have control over which organisation we source the data from, and how many forecast/historical days of data we want to collect.

Data sourced from:
- https://open-meteo.com/en/docs/bom-api

## Install packages and import libraries

In [1]:
%pip install openmeteo-requests -q #open-meteo is free for non-commercial use
%pip install requests-cache -q
%pip install retry-requests -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# OS packages
import requests_cache
from retry_requests import retry
from datetime import datetime, time, timedelta

# open-meteo
import openmeteo_requests
import pandas as pd
import geopandas as gpd

## What data can we get from Open Meteo and how does it compare to EOS?

- add comparison to EOS and list side-by-side table


### Current/ forecast weather
Open Meteo lets the user select from XXX government weather models, documented in the API:

(put link to open meteo here)

This demonstration uses the Beureau of Meteorology model.


- aggregate request only needs geojson and gives 3 day forecast out from request date/time
- request without data ggregation takes a start and end date in addition to the geosjon

### Historical Weather

Open Meteo also lets suers get historical, actual weather data (forecast data is modelled, historical data is measured)

## Set up directories, cache, input geometry

For working locally, an input and output directory are used. This can be replaced with Nav's S3 Utilities later on.

A cache is set up so that, if the same request is made within a time limit (1 hour) a new API call won't be made if there is no difference from the previous request.

The geojson for Yarra Yarra is used in this demo, and is also imported. The central latitude and longitude (centre point of the polygon) is what is used in the Open Meteo API request.

In [3]:
input_root_dir = '/workspace/notebooks/sandbox/data/input-data/open-meteo'
output_root_dir = '/workspace/notebooks/sandbox/data/output-data/open-meteo'

In [4]:
# Setup the Open-Meteo API client with cache and retry on error
cache_dir = input_root_dir + '/.cache'
cache_session = requests_cache.CachedSession(cache_dir, expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [5]:
# Read in test area and get bounding box and lat/lon
#file_path = '/workspace/notebooks/sandbox/data/input-data/dissolved-boundaries.geojson'

file_path = 'dissolved-boundaries.geojson'
geom = gpd.read_file(file_path)

# get the central latitude and longitude of the polygon
gpd_lon = geom.centroid.x[0]
gpd_lat = geom.centroid.y[0]

/tmp/ipykernel_426/3517518904.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gpd_lon = geom.centroid.x[0]
/tmp/ipykernel_426/3517518904.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gpd_lat = geom.centroid.y[0]


In [6]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below

# for wint speed, direction etc. the 10m and 40m are metres above surface level, not a cap on the maximum values recordable.
url = "https://api.open-meteo.com/v1/bom"
params = {
	"latitude": gpd_lat,
	"longitude": gpd_lon,
	"hourly": ["temperature_2m", 
            "relative_humidity_2m", 
            "precipitation", 
            "rain", 
            "surface_pressure", 
            "cloud_cover", ###
            "visibility", ###
            "wind_speed_10m", 
            "wind_direction_10m",
            "soil_temperature_0_to_10cm", ###
            "soil_temperature_10_to_35cm"], ###
 	"past_days": 3,
	"forecast_days": 3
}

responses = openmeteo.weather_api(url, params=params)

In [7]:
# # Process first location. Add a for-loop for multiple locations or weather models

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates -29.23828125°N 116.279296875°E
Elevation 294.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [8]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
hourly_visibility = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_temperature_0_to_10cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_temperature_10_to_35cm = hourly.Variables(10).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}


hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["visibility"] = hourly_visibility
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["soil_temperature_0_to_10cm"] = hourly_soil_temperature_0_to_10cm
hourly_data["soil_temperature_10_to_35cm"] = hourly_soil_temperature_10_to_35cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.head()

,date,temperature_2m,relative_humidity_2m,precipitation,rain,surface_pressure,cloud_cover,visibility,wind_speed_10m,wind_direction_10m,soil_temperature_0_to_10cm,soil_temperature_10_to_35cm
0,2024-03-18 00:00:00+00:00,20.296000,75.0,0.0,0.0,976.703674,36.0,12400.0,13.320000,194.0,23.945999,30.146000
1,2024-03-18 01:00:00+00:00,22.496000,63.0,0.0,0.0,976.661072,16.0,13540.0,15.119999,187.0,23.896000,29.945999
2,2024-03-18 02:00:00+00:00,24.846001,54.0,0.0,0.0,976.824219,0.0,15180.0,14.759999,183.0,24.646000,29.896000
3,2024-03-18 03:00:00+00:00,27.546000,43.0,0.0,0.0,976.537720,0.0,16520.0,15.840000,180.0,25.896000,29.746000
4,2024-03-18 04:00:00+00:00,29.396000,37.0,0.0,0.0,976.445801,0.0,17040.0,17.280001,182.0,27.246000,29.746000


In [9]:
#hourly_dataframe.to_csv(output_root_dir + '/yarrayarra_bom_data.csv', index = False)

hourly_dataframe.to_csv('yarrayarra_bom_data.csv', index = False)